## Breakout Atari 2600

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
#import matplotlib as plt
import matplotlib.pyplot as plt

#!conda install -c conda-forge gym
import gym

In [2]:
tf.random.set_seed(42)
np.random.seed(42)

#!pip install tf-agents

# Reinforcement Learning Breakout Atari 2600

## 1 - Preparar el Ambiente

Vamos a crear una clase AtariPreprocessingWithAutoFire ya que el juego ocupa disparar en el primer frame para iniciar el juego, asi que apenas se haga reset, este inicial el juego con una accion Fire.

El ambiente a usar es **"BreakoutNoFrameskip-v4"** ya que este no usa max polling ni frame skipping.

In [3]:
from tf_agents.environments import suite_atari
from tf_agents.environments.atari_preprocessing import AtariPreprocessing
from tf_agents.environments.atari_wrappers import FrameStack4 # Stack 4 frames previos

max_episode_steps = 27000 # <=> 108k ALE frames ya que 1 step = 4 frames
environment_name = "BreakoutNoFrameskip-v4"

class AtariPreprocessingWithAutoFire(AtariPreprocessing):
    def reset(self, **kwargs):
        obs = super().reset(**kwargs)
        super().step(1) # Start
        return obs
    def step(self, action):
        lives_before_action = self.ale.lives()
        obs, rewards, done, info = super().step(action)
        if self.ale.lives() < lives_before_action and not done:
            super().step(1) # Auto Start si hay una muerte. (se pifea la bolita)
        return obs, rewards, done, info

# vamos a crear el ambiente con 2 wrappers, uno custom, y otro llamado FrameStack4   
env = suite_atari.load(
    environment_name,
    max_episode_steps=max_episode_steps,
    gym_env_wrappers=[AtariPreprocessingWithAutoFire, FrameStack4])

In [4]:
# vamos a convertir el ambiente actual en un ambiente de Tensorflow (como un grafo de tensorflow).

from tf_agents.environments.tf_py_environment import TFPyEnvironment

tf_env = TFPyEnvironment(env)

### DQN

Vamos a crear una clase pequeña para normalizar las observaciones. Las imágenes se almacenan usando bytes de 0 a 255 para usar menos RAM, pero queremos pasar flotantes de 0.0 a 1.0 a la red neuronal:

In [5]:
from tf_agents.networks.q_network import QNetwork

# normaliza 0.0 a 1.0
preprocessing_layer = keras.layers.Lambda(
                          lambda obs: tf.cast(obs, np.float32) / 255.)

# capas connvoluciuonales con los filtros a utilizar
conv_layer_params=[(32, (8, 8), 4), (64, (4, 4), 2), (64, (3, 3), 1)]

# capa Dense(512) despues salen 4 valores de output (Q-Values)
fc_layer_params=[512]

# Deep Q-Network (Q-Learning)
# esta red devuelve 1 Q-Value por cada accion. (recordemos son 4)
q_net = QNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(), # 4 acciones
    preprocessing_layers=preprocessing_layer,
    conv_layer_params=conv_layer_params,
    fc_layer_params=fc_layer_params)

### Definicion del DqnAgent

In [6]:
from tf_agents.agents.dqn.dqn_agent import DqnAgent

train_step = tf.Variable(0)
update_period = 4 # un training step recolecta 4 pasos
optimizer = keras.optimizers.RMSprop(lr=2.5e-4, rho=0.95, momentum=0.0,
                                     epsilon=0.00001, centered=True)

# funcion que calcula e y e-greedy
epsilon_fn = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1.0, # initial ε
    decay_steps=250000 // update_period, # <=> 1,000,000 ALE frames
    end_learning_rate=0.01) # final ε

# el agente con la DQN creada y todos los parametros necesarios
agent = DqnAgent(tf_env.time_step_spec(),
                 tf_env.action_spec(),
                 q_network=q_net,
                 optimizer=optimizer,
                 target_update_period=2000, # <=> 32,000 ALE frames
                 td_errors_loss_fn=keras.losses.Huber(reduction="none"),
                 gamma=0.99, # factor de descuento
                 train_step_counter=train_step,
                 epsilon_greedy=lambda: epsilon_fn(train_step))

# se inicializa el agente
agent.initialize()

### Creamos el Replay Buffer

In [7]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=tf_env.batch_size,
    max_length=100000) # reducir si da Out-of-Memory Error (OOM)

### Creamos el Observador

In [8]:
replay_buffer_observer = replay_buffer.add_batch

### Funciones Utilitarias

In [9]:
class ShowProgress:
    def __init__(self, total):
        self.counter = 0
        self.total = total
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            print("\r{}/{}".format(self.counter, self.total), end="")

In [10]:
from tf_agents.metrics import tf_metrics

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric(),
]

train_metrics[0].result()

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [11]:
from tf_agents.eval.metric_utils import log_metrics
import logging
logging.getLogger().setLevel(logging.INFO)
log_metrics(train_metrics)

INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 0
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0


### Creamos el Driver

In [12]:
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver

collect_driver = DynamicStepDriver(
    tf_env,
    agent.collect_policy,
    observers=[replay_buffer_observer] + train_metrics,
    num_steps=update_period) # recolecte 4

### Creamos El Collect Policy

El collect driver es quien explora el ambiente, dada unas politicas y luego manda esta informacion al observer, quien luego pasa esa informacion al Replay Buffer.

In [13]:
from tf_agents.policies.random_tf_policy import RandomTFPolicy

initial_collect_policy = RandomTFPolicy(tf_env.time_step_spec(),
                                        tf_env.action_spec())
# DynamicStepDriver
# -  recolecta experiencias cada n numero de steps

init_driver = DynamicStepDriver(
    tf_env,
    initial_collect_policy,
    observers=[replay_buffer.add_batch, ShowProgress(20000)],
    num_steps=20000) # <=> 80,000 ALE frames

# ejecutamos el Collect Policy
final_time_step, final_policy_state = init_driver.run()

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


20000/20000

### Crear el DataSet

In [14]:
dataset = replay_buffer.as_dataset(
    sample_batch_size=64,
    num_steps=2,
    num_parallel_calls=3).prefetch(3) # pre-carga 3 batches

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [15]:
# algunos wraps con function para usar en el main loop...
from tf_agents.utils.common import function

collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

### Main Loop (Entrenamiento del Agente)

In [16]:
def train_agent(n_iterations):
    time_step = None
    policy_state = agent.collect_policy.get_initial_state(tf_env.batch_size)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        print("\r{} loss:{:.5f}".format(
            iteration, train_loss.loss.numpy()), end="")
        if iteration % 1000 == 0:
            log_metrics(train_metrics)
            
train_agent(n_iterations=50000) # que tal 10,000,000 ? mejor que un humano? intentelo....

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 4
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0


999 loss:0.00003

INFO:absl: 
		 NumberOfEpisodes = 27
		 EnvironmentSteps = 4004
		 AverageReturn = 0.699999988079071
		 AverageEpisodeLength = 144.39999389648438


1995 loss:0.00004

...

47996 loss:0.00054

INFO:absl: 
		 NumberOfEpisodes = 1056
		 EnvironmentSteps = 192004
		 AverageReturn = 6.0
		 AverageEpisodeLength = 303.5


48996 loss:0.00037

INFO:absl: 
		 NumberOfEpisodes = 1068
		 EnvironmentSteps = 196004
		 AverageReturn = 8.600000381469727
		 AverageEpisodeLength = 341.1000061035156


49999 loss:0.00131

In [17]:
from matplotlib.animation import FuncAnimation

def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=40):
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    anim = FuncAnimation(
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim

In [18]:
%matplotlib inline
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

frames = []
def save_frames(trajectory):
    global frames
    frames.append(tf_env.pyenv.envs[0].render(mode="rgb_array"))

watch_driver = DynamicStepDriver(
    tf_env,
    agent.policy,
    observers=[save_frames, ShowProgress(1000)],
    num_steps=1000)
final_time_step, final_policy_state = watch_driver.run()

plot_animation(frames)

900/1000

INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


1000/1000